In [83]:
import os;

import sys;
sys.path.insert(0,os.getcwd() + '\\..\\lib');

import numpy as np
import scipy.io as spio;
from random import randint;
import matplotlib.pyplot as plot

data1 = spio.loadmat("D:/Codes/machine learning coursera/ex4/ex4data1.mat");
X = data1["X"];
y = data1["y"];

def displayImage(X):
    
    # fname = 'image.png'
    # image = Image.open(fname).convert("L")
    # arr = np.asarray(image)

    plot.imshow(X, cmap='gray')
    plot.show()

# displayImage(np.reshape(X[0],(20,20)));

data1 = spio.loadmat("D:/Codes/machine learning coursera/ex4/ex4weights.mat");
theta1 = data1["Theta1"];
theta2 = data1["Theta2"];
class NeuralNetwork:
    
    def __init__(self, X, y, lam, hiddenLayerSizes):
        self.X = X;
        self.y = y;
        self.lam = lam;
        self.m = X.shape[0];
        self.n = X.shape[1];
        self.X = NeuralNetwork.addIntercept(self.X);
        print(
        
            [n] + hiddenLayerSizes.append(yClasses)
            
        );
        thetasDimensions = [n] + hiddenLayerSizes.append(yClasses);
        for i in range(0, len(thetaDimensions)-1):
            self.Thetas.append(self.initializeRandomWeights(thetaDimensions[i+1], thetaDimensions[i]+1));
        print(
        
            self.Thetas
        
        );

    @staticmethod
    def addIntercept(X):
        m = X.shape[0];
        X = np.hstack((np.ones((m,1)), X));
        return X;       
    
    @staticmethod
    def sigmoidGradient(z):
        return np.multiply(LR.sigmoid(z), np.subtract( 1, LR.sigmoid(z)));
    
    @staticmethod
    def initializeRandomWeights(lIn, lOut):
        epsilonInit = 0.12;
        w = np.zeros((lOut, lIn + 1));
        w = np.random.rand(lOut, lIn + 1);
        w = np.subtract( np.multiply( w, (2.0 * epsilonInit) ), epsilonInit );
        return w;
    
    
    
    def costFunction(theta, n, hiddenLayerSize, yClasses, X, y, lam):

        m = X.shape[0];
        
        startIndex = 0;
        a = X;
        self.a = [];
        self.z = [];
        regularizationSum = 0;
        
        for i in range(0, len(self.Thetas)):
            endIndex = self.Thetas[i].shape[0] * self.Thetas[i].shape[0];
            self.Thetas[i] = np.reshape(nn_params[startIndex:endIndex, self.Thetas[i].shape);
            startIndex = endIndex;
            
            z = np.dot(self.Thetas[i], np.transpose(a))
            self.z.append(z);
            a = NeutralNetwork.sigmoid(z);
            self.a.append(a);
      
            regularizationSum += np.sum(
                    np.square(
                        np.hstack( [np.zeros((self.Thetas[i].shape[0],1)), self.Thetas[i][:, 1:]] )
                    )
            );
                                                  
        J = np.add(
                np.multiply(
                    (1/self.m) 
                    , np.sum( 
                            np.subtract(
                                np.multiply(
                                    np.multiply(-1, self.y),
                                    np.log(a)
                                )
                                , np.multiply(
              
                                    np.subtract(1, self.y), 
                                    np.log(np.subtract(1, a))
                                )
                            )
                        )
                   )  
                , (
                    (self.lam/(2 * self.m))  
                    * regularizationSum
                )
        );
                                                  
        
#         theta2 = np.reshape(nn_params[ (hiddenLayerSize * (n+1)): ], theta2Shape);
        
        
        a1 = np.transpose(LR.addIntercept(X));
        z2 = np.dot(theta1, a1);
        a2 = LR.sigmoid(z2);
        a2 = np.transpose(LR.addIntercept(np.transpose(a2)));
        z3 = np.dot(theta2, a2);
        a3 = LR.sigmoid(z3);
                                                  
        y1 = np.zeros((m, yClasses));

        for i in range(0,m):
            y1[i, y[i] % 10] = 1;

        delta3 = np.subtract(a3, np.transpose(y1));
        delta2 = np.multiply( np.dot(np.transpose(theta2), delta3)[1:, :], sigmoidGradient(z2));

        theta1Grad = np.zeros(theta1Shape);
        theta2Grad = np.zeros(theta2Shape);

        theta1Grad = theta1Grad + np.dot(delta2, np.transpose(a1));
        theta2Grad = theta2Grad + np.dot(delta3, np.transpose(a2));  

        theta1Grad = np.divide(theta1Grad, m) + np.hstack( [np.zeros((theta1.shape[0],1)), np.multiply((lam/m), theta1[:, 1:])] );
        theta2Grad = np.divide(theta2Grad, m) + np.hstack( [np.zeros((theta2.shape[0],1)), np.multiply((lam/m), theta2[:, 1:])] );

    def GradFunction(nn_params, n, hiddenLayerSize, yClasses, X, y, lam):
    
m = X.shape[0];
n = X.shape[1];
hiddenLayerSize = 25;
yClasses = 10;
lam = 0;

theta1 = initializeRandomWeights(n, hiddenLayerSize);
theta2 = initializeRandomWeights(hiddenLayerSize, yClasses)

nn_params = np.hstack([theta1.flatten(), theta2.flatten()])
print(
    theta1.shape,
    theta2.shape,
    nn_params.shape
    
)
J = costFunction(nn_params, n, hiddenLayerSize, yClasses, X, y, lam);

fminOutput = fmin_bfgs(
                            costFunction,
                            nn_params,
                            gradFunction,
                            disp=True,
                            maxiter=400,
                            full_output = True,
                            retall=True
                        );

IndentationError: expected an indented block (<ipython-input-83-69045e1395ac>, line 141)

In [8]:
test = [25, 20]
[401, test, 10]
test.append(10)
[401] + test

[401, 25, 20, 10]